In [0]:
#!import "../DataModel/DataStructure"

In [0]:
public record CashflowIdentity{
    public string DataNode {get; set;}
    public string AmountType {get; set;}
    public int? AccidentYear {get; set;}
}

# Storage

In [0]:
public class CashflowStorage
{   
    private readonly IDataSource querySource; 
    private readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
    private readonly ImportArgs args;

    //Dimensions
    public HashSet<string> EarningsAmountTypes  { get; private set; }
    public HashSet<string> SolvencyBsAmountTypes  { get; private set; }
    public HashSet<string> IfrsBsAmountTypes  { get; private set; }

    public Cashflow[] CashflowsByIdentity { get; private set; }
    public BasicCashflow[] BasicCashflowsByIdentity { get; private set; }
    public BoundedCashflow[] BoundedCashflowsByIdentity { get; private set; }
    public ReferencedCashflow[] ReferencedCashflowsByIdentity { get; private set; }

       
    public CashflowIdentity[] Identities {get; set;}

    // Constructor
    public CashflowStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }
    
    // Initialize
    public async Task InitializeAsync()
    {   
        // Dimensions
        EarningsAmountTypes = (await workspace.Query<EarningsAmountType>().ToArrayAsync()).Select(x => x.SystemName).ToHashSet();
        SolvencyBsAmountTypes = (await workspace.Query<SolvencyBsAmountType>().ToArrayAsync()).Select(x => x.SystemName).ToHashSet();
        IfrsBsAmountTypes = (await workspace.Query<IfrsBsAmountType>().ToArrayAsync()).Select(x => x.SystemName).ToHashSet();
        
        // Identities
        CashflowsByIdentity = await workspace.Query<Cashflow>().ToArrayAsync();
        BasicCashflowsByIdentity = await workspace.Query<BasicCashflow>().ToArrayAsync();
        BoundedCashflowsByIdentity = await workspace.Query<BoundedCashflow>().ToArrayAsync();
        ReferencedCashflowsByIdentity = await workspace.Query<ReferencedCashflow>().ToArrayAsync();

        Identities = CashflowsByIdentity.Select(x => new CashflowIdentity() 
            { 
                DataNode = x.DataNode,
                AmountType = x.AmountType,
                AccidentYear = x.AccidentYear
            }
        ).Distinct().ToArray();

    }

   
   // Getters
    public IEnumerable<BasicCashflow> GetBasicCashflows(CashflowIdentity identity) => 
        BasicCashflowsByIdentity.Where(x => x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);


    public IEnumerable<BoundedCashflow> GetBoundedCashflows(CashflowIdentity identity) => 
        BoundedCashflowsByIdentity.Where(x => x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);
    
    public IEnumerable<ReferencedCashflow> GetReferencedCashflow(CashflowIdentity identity) => 
        ReferencedCashflowsByIdentity.Where(x => x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);

}